In [1]:
## Делает графики на маке четче
%config InlineBackend.figure_format='retina'

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import re
import os 
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Обработка
## Загрузка данных

In [4]:
ROOT_DIR = "/Users/pus/ML/Лаборатория/СеверСталь/Probe/"
DIR_PATH = "Probe_Data/"
for filename in os.listdir(ROOT_DIR + DIR_PATH):
    print(filename)

Февраль.xlsx
.DS_Store
SensorsData.csv
10 фераля _избранные сигналы.txt
Febraury_PredData.csv
2020_02_10_PrepData.csv
Febraury_StopLog.csv
.ipynb_checkpoints


In [5]:
df = pd.read_csv(''.join([ROOT_DIR, DIR_PATH ,"10 фераля _избранные сигналы.txt"]), sep=';', encoding='cp1251')

In [6]:
df.values[0,1:-2].tolist()

['А15_СH0_IW128_Температура обмотки статора А (датчик №1)',
 'А15_СH1_IW130_Температура обмотки статора А (датчик №2)',
 'А15_СH2_IW132_Температура обмотки статора В (датчик №1)',
 'А15_СH3_IW134_Температура обмотки статора В (датчик №2)',
 'А15_СH4_IW136_Температура обмотки статора С (датчик №1)',
 'А15_СH5_IW138_Температура обмотки статора С (датчик №2)',
 'А15_СH6_IW140_Температура переднего подшипника двигателя',
 'А15_СH7_IW142_Температура заднего подшипника двигателя',
 'А16_СH0_IW144_Давление в масляном домкрате (передний подшипник)',
 'А16_СH1_IW146_Давление в масляном домкрате (задний подшипник)',
 'А16_СH2_IW148_Вибрация переднего подшипника двигателя (осевая)',
 'А16_СH3_IW150_Вибрация переднего подшипника двигателя (радиальная)',
 'А16_СH4_IW152_Вибрация заднего подшипника двигателя (осевая)',
 'А17_СH1_IW162_Расход маслостанции',
 'А17_СH2_IW164_Температура в баке маслостанции',
 'А17_СH4_IW168_Скорость двигателя ПЧ БМЗ-1',
 'А17_СH5_IW170_Ток двигателя ПЧ БМЗ-1',
 'DB41_f

In [7]:
columns = df.values[0,1:-2].tolist()
ad_col = df.values[0,-2].split(';')
columns+=ad_col
columns = list(map(lambda s: re.sub(r'[A-z]', '', s)[8:], columns))

time_index = pd.to_datetime(df.Time[1:])

In [8]:
Data = pd.DataFrame(df.values[1:,1:], columns=columns, index=time_index.values)
Data.iloc[:,:] = np.array(list(map(lambda x: float(str(x).replace(',','.')), Data.values[:,:].ravel()))).reshape(-1,22)


In [9]:
Data.to_csv("2020_02_10_PrepData.csv", index=1)

## Функции визуализации

In [10]:
from copy import deepcopy
def plot_melting_process(axes, Left_Border, Right_Border, MAX, MIN, Melting_args):
    '''
    В качестве аргументов передается массив из Bool значений
    Если True - рисуется соответсвующий (ниже список) график
    процесса
    
    ' Плавка Завалка скрапа' 
    ' Плавка Завалка чугуна'
    ' Плавка Продувка'
    ' Плавка Слив стали'
    ' Плавка Слив шлака'
    '''
    
    
    if Melting_args[0]:
        axes.plot((Data.iloc[Left_Border:Right_Border][' Плавка Завалка скрапа'])*(MAX-MIN)+MIN, 
                 c='r', alpha=0.6, label = 'Завалка скрапа')
    if Melting_args[1]:
        axes.plot((Data.iloc[Left_Border:Right_Border][' Плавка Завалка чугуна'])*(MAX-MIN)+MIN, 
                 c='c', alpha=0.6, label = 'Завалка чугуна')
    if Melting_args[2]:
        axes.plot((Data.iloc[Left_Border:Right_Border][' Плавка Продувка'])*(MAX-MIN)+MIN, 
                 c='m', alpha=0.6, label = 'Продувка')
    if Melting_args[3]:
        axes.plot((Data.iloc[Left_Border:Right_Border][' Плавка Слив стали'])*(MAX-MIN)+MIN, 
                 c='y', alpha=0.6, label = 'Слив стали')
    if Melting_args[4]:
        axes.plot((Data.iloc[Left_Border:Right_Border][' Плавка Слив шлака'])*(MAX-MIN)+MIN, 
                 c='k', alpha=0.4, label = 'Слив шлака')
    plt.legend(fontsize=12, loc='lower left')

In [11]:
@interact
def plot(Location = (1000,85000,1000),
         Width = (1000,50000,1000),
         Signal_1 = Data.columns[:-5],
         Signal_2 = Data.columns[:-5],
         Завалка_Скрапа = False,
         Завалка_Чугуна = False,
         Продувка = False,
         Слив_Стали = False,
         Слив_Шлака = False):
    
    fig, ax1 = plt.subplots(figsize=(15,9))
    Left_Border = max(0, Location-Width)
    Right_Border = min(85001, Location+Width)    
    
    ax1.tick_params(axis='y', labelcolor='b')    
    ax1.plot(Data.iloc[Left_Border:Right_Border][Signal_1], c='b', label=Signal_1)
    
    ax2 = ax1.twinx()
    ax2.tick_params(axis='y', labelcolor='orange')
    ax2.set_yticks
    ax2.plot(Data.iloc[Left_Border:Right_Border][Signal_2], c='orange', label=Signal_2)
    
    MAX_1 = Data.iloc[Left_Border:Right_Border][Signal_1].max()
    MIN_1 = Data.iloc[Left_Border:Right_Border][Signal_1].min()
    MAX_2 = Data.iloc[Left_Border:Right_Border][Signal_2].max()
    MIN_2 = Data.iloc[Left_Border:Right_Border][Signal_2].min()
    
    Melting_Args = [Завалка_Скрапа, Завалка_Чугуна, Продувка, Слив_Стали, Слив_Шлака]
    plot_melting_process(ax1, Left_Border, Right_Border, MAX_1, MIN_1, Melting_Args)
    
    ax1.set_ylim(top=MAX_1+0.4*(MAX_1-MIN_1))
    ax2.set_ylim(top=MAX_2+0.4*(MAX_2-MIN_2))
    
    ax1.legend(fontsize=12, loc='upper left')
    ax2.legend(fontsize=12, loc='upper right')
    ax1.grid()
    plt.show()

interactive(children=(IntSlider(value=43000, description='Location', max=85000, min=1000, step=1000), IntSlide…

In [12]:
# standard bokeh imports
from bokeh.io import output_notebook, show, reset_output

# other bokeh imports
import bokeh
from bokeh.plotting import figure
# more imports in cells below as needed

# other imports
import numpy as np
import pandas as pd

In [13]:
output_notebook()

Loading BokehJS ...

In [15]:
from bokeh.models import HoverTool, Range1d, LinearAxis
from copy import deepcopy


@interact
def plot(Завалка_Скрапа = False,
         Завалка_Чугуна = False,
         Продувка = False,
         Слив_Стали = False,
         Слив_Шлака = False,
         Signal_1 = Data.columns[:-5],
         Signal_2 = Data.columns[:-5]):
    
    df = deepcopy(Data.iloc[:84000:60,:])
    line_plot = figure(plot_width=950, plot_height=600, title='{} и {}'.format(Signal_1, Signal_2),
                       x_axis_type="datetime", toolbar_location="above",
                       active_scroll="wheel_zoom")
    
    y_overlimit = 0.05
    
    ## Signal 1 Plot
    
    Signal_1_range = "Signal_1_range"
    Signal_2_range = "Signal_2_range"
    
    line_plot.extra_y_ranges = {
        Signal_1_range: Range1d(df[Signal_1].min() * (1 - y_overlimit),
                                df[Signal_1].max() * (1 + y_overlimit)),
        
        Signal_2_range: Range1d(df[Signal_2].min() * (1 - y_overlimit),
                                df[Signal_2].max() * (1 + y_overlimit))
    }
#     line_plot.y_range = Range1d(df[Signal_1].min() * (1 - y_overlimit),
#                                 df[Signal_1].max() * (1 + y_overlimit))
    line_plot.line(x=df.index,
                   y=df[Signal_1],
                   legend=Signal_1,
                   line_width=2,
                   y_range_name=Signal_1_range,
                   color="red",)

    ## Signal 2 Plot


    line_plot.add_layout(LinearAxis(y_range_name=Signal_2_range), "right")

    line_plot.line(x=df.index,
                   y=df[Signal_2],
                   legend=Signal_2,
                   line_width=2,
                   y_range_name=Signal_2_range,
                   color="green",)
    
    MIN = min(df[Signal_1].min(), df[Signal_2].min())
    MAX = max(df[Signal_1].max(), df[Signal_2].max())
    Melting_args = [Завалка_Скрапа, Завалка_Чугуна, Продувка, Слив_Стали, Слив_Шлака]
    
    df = df*MAX
    df['date_index'] = df.index
    
    _fa = 0.15
    if Melting_args[0]:
        line_plot.varea_stack([' Плавка Завалка скрапа'],
                               x='date_index',
                               color=('magenta'),
                               source=df,
                               fill_alpha=_fa,
                               legend='Плавка Завалка скрапа')

    if Melting_args[1]:
        line_plot.varea_stack([' Плавка Завалка чугуна'],
                               x='date_index',
                               color=('orange'),
                               source=df,
                               fill_alpha=_fa,
                               legend='Плавка Завалка чугуна')

    if Melting_args[2]:
        line_plot.varea_stack([' Плавка Продувка'],
                               x='date_index',
                               color=('red'),
                               source=df,
                               fill_alpha=_fa,
                               legend='Плавка Продувка')

    if Melting_args[3]:
        line_plot.varea_stack([' Плавка Слив стали'],
                               x='date_index',
                               color=('green'),
                               source=df,
                               fill_alpha=_fa,
                               legend='Плавка Слив стали')

    if Melting_args[4]:
        line_plot.varea_stack([' Плавка Слив шлака'],
                               x='date_index',
                               color=('blue'),
                               source=df,
                               fill_alpha=_fa,
                               legend='Плавка Слив шлака')
    
    # add hover tool
    line_plot.add_tools(HoverTool())


    # another way to set axis labels
    # line_plot.xaxis.axis_label = 'x-axis'
    # line_plot.yaxis.axis_label = 'y-axis'

    show(line_plot)

interactive(children=(Checkbox(value=False, description='Завалка_Скрапа'), Checkbox(value=False, description='…

# Вопросы

* Накануне критической остановкой в 10:55, профиль осевой вибрации переднего подшипника двигателя сильно изменяется. Связано ли это с нетипичной последовательностью завалки, продувки, слива?

* Почему вибрация резко увеличивается в момент начала продувки а не в момент завалки скрапа? Как именно осуществляется продувка?
> Потому что мы нагнетатель изучаем, он как раз продувку и осуществляет

* Правильно ли я понял, что весь процесс - это по сути процесс переплавки? Если нет, то какова роль скрапа?
> Весь процесс это переплавка чугуна в сталь. Чугун из домменой печи или типа того получается, потом его закидывают в конвертер, чтобы от шлаков и примесей очистить, как я понимаю

# _Заметки_

> _Для обмоток статора A, B, C показания датчиков №1 и №2 практически не отличаются (дублируют друг друга)_

> _Заметил, что периодически линии спадающего фронта этапов плавки совпадают с точками изгиба показаний датчика температуры обмотки статора_

> В определенное время, например с 08:00 до 11:00 `(loc=34000, width=6000)` можно заметить нетипичное поведение фронта осевой вибрации переднего подшипника двигателя во время продувки
